In [ ]:
import hw3_functions as hw3_func

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import normalize
from numpy import linalg as LA
import collections

In [ ]:
#read input data
data_path = "/home/hs/study/cse_601/data/Data_For_HW3/"
#input_file = "yeast_undirected_metabolic.txt"
#input_file = "physics_collaboration_net.txt"
#input_file = "beta_test_data.txt"
input_file  = "attweb_net.txt"
#input_file = 'example.txt'
ip_file = data_path + input_file
input_delimiter = " "
ip = pd.read_table(filepath_or_buffer=ip_file,sep=input_delimiter,header=None)
if(input_file=="attweb_net.txt"):
    pass
else:
    ip,dict_map = hw3_func.ip_df_serialised(ip)

In [ ]:
#set parameters for running the program
max_iter = 50
#assuming serial numbering of nodes
node_min = ip.min(axis=0).min()
node_max = ip.max(axis=0).max()
node_range = node_max - node_min
single_value_run = False
#expansion range 2-5, steps of 1
#inflation range 1.1-2, steps of 0.1
init_expansion_param = 2
init_inflation_param = 1.0
step_expansion_param = 1
step_inflation_param = 0.1
max_inflation_param = 2
max_expansion_param = 5
ip_copy = ip.loc[:,:]

In [ ]:
#create init_matrix
init_matrix = np.eye(N=node_range+1)
#populate initial_matrix
hw3_func.populate_init_matrix(init_matrix=init_matrix,input_dataframe=ip)
#normalize initial_matrix
norm_matrix = hw3_func.normalize_matrix_column(matrix_tb_norm=init_matrix)
#set Prev matrix for while loop check
prev_matrix = init_matrix
#create safe copy
new_norm_matrix = norm_matrix[:,:]
cur_expansion_param = init_expansion_param
cur_inflation_param = init_inflation_param
#create df to store clustering results
if(single_value_run==True):
    col_list = ['exp_'+str(init_expansion_param)+'_infl_'+str(init_inflation_param)]
    df_dummy_fill = np.zeros(shape=(node_range+1,len(col_list)),dtype=np.int64)
    result_df = pd.DataFrame(data=df_dummy_fill,columns=col_list,index=np.arange(0,node_range+1))
    max_expansion_param = init_expansion_param
    max_inflation_param = init_inflation_param
else:
    exp_arr = np.arange(cur_expansion_param,max_expansion_param+1,step_expansion_param)
    infl_arr = np.arange(cur_inflation_param,max_inflation_param+step_inflation_param,step_inflation_param)
    col_list = []
    for exp_val in exp_arr:
        for infl_val in infl_arr:
            col_list.append('exp_'+str(exp_val)+'_infl_'+str(infl_val))
    df_dummy_fill = np.zeros(shape=(node_range+1,len(col_list)),dtype=np.int64)
    result_df = pd.DataFrame(data=df_dummy_fill,columns=col_list,index=np.arange(0,node_range+1))

In [ ]:
while((cur_expansion_param<=max_expansion_param)):
    #initilize counter
    num_iter=0
    #adding rounding off
    cur_inflation_param=round(cur_inflation_param,1)
    #initialize list
    #set Prev matrix for while loop check
    prev_matrix = init_matrix
    #create safe copy
    new_norm_matrix = norm_matrix[:,:]
    #print('Current expansion/inflation params : {}/{}'.format(cur_expansion_param,cur_inflation_param))
    #while loop where all the action happens
    while((num_iter<max_iter) and \
          (not((hw3_func.check_matrix_di(matrix_cur=new_norm_matrix,matrix_prev=prev_matrix))))):
        prev_matrix=new_norm_matrix[:,:]
        new_norm_matrix = hw3_func.expand_inflate_normalize(matrix_input=new_norm_matrix,expansion_param=cur_expansion_param\
                                                   ,inflation_param=cur_inflation_param,prune_by_rounding=False)
        num_iter+=1
        #print(num_iter)
    hw3_func.extract_2_df_cluster_grouping(stable_matrix=new_norm_matrix,exp_val=cur_expansion_param,\
                                 infl_val=cur_inflation_param,result_df=result_df,min_positive_value=0.0)
    #increment cur_inflation_param
    cur_inflation_param += step_inflation_param
    if(cur_inflation_param >max_inflation_param):
        cur_inflation_param = init_inflation_param
        cur_expansion_param+=step_expansion_param

In [ ]:
hw3_func.write_result(input_file=input_file,result_df=result_df)